In [1]:
import numpy as np
import torch
print("Torch version:", torch.__version__)
from torchvision import transforms, utils
from barbar import Bar
from parse_image import *
import clip
import torch
from PIL import Image
GPU = 0
data_path = "path/to/csv.csv" ## ComVG or SVO_Probes path

Torch version: 2.0.1+cu117


In [3]:
#Replace with correct model version
model, preprocess = clip.load("RN50", device='cpu')
model.cuda(GPU).eval()

In [ ]:
import pandas as pd
data = pd.read_csv(data_path)
data.head(1)

In [6]:
clip_scores = []
for idx, row in data.iterrows():
    text_tokens = clip.tokenize([row.sentence]).cuda(GPU)
    image_pos, image_neg = preprocess(read_image(row.pos_image_id)), preprocess(read_image(row.neg_image_id))
    images = [image_pos, image_neg]
    image_input = torch.tensor(np.stack(images)).cuda(GPU)
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        text_features = model.encode_text(text_tokens).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
    clip_scores.append({"id":idx, "pos_score": similarity[0][0], "neg_score": similarity[0][1]})

In [ ]:
all_verb = 0
verb_faile = 0 
acc = 0
for i in clip_scores:
    if i["pos_score"] > i["neg_score"]:
        acc += 1
acc / len(clip_scores)